In [1]:
import os
import shutil

directory_out = './file/5.collate'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

from tqdm import tqdm
import json

# 1. Part

In [2]:
directory_in = './file/3.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =

    df_input = pd.read_excel(f'{directory_in}/{file}',
                             header=0,
                             dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    # = = = = = = = = = = = = = = =

    list_series, list_column, list_column_specification = [], df_input.columns.tolist(), []
    for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
        dict_dict = json.loads(series['Json_Specification'])
        dict_ = {k: v for dict_ in dict_dict.values() for k, v in dict_.items()}

        for key, value in dict_.items():
            if key in list_column:
                key = f'{key} (Specification)'

            if key not in list_column_specification:
                list_column_specification.append(key)

            series[key] = value

        list_series.append(series)

    # = = = = = = = = = = = = = = =

    df_output = pd.DataFrame(list_series).sort_values(by=['No'],
                                                      ascending=[True],
                                                      ignore_index=True)
    df_output[list_column[:9] + sorted(list_column_specification) + list_column[9:]].to_excel(f'{directory_out}/{file}', index=False)

print()
print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 93/93 [00:00<00:00, 287.81it/s]


Done ~


# 2. Vehicle

In [3]:
directory_in = './file/4.vehicle'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =

    df_input = pd.read_excel(f'{directory_in}/{file}',
                             header=0,
                             dtype=str).fillna('')

    df_input['Page'] = df_input['Page'].astype(int)
    df_input['Row'] = df_input['Row'].astype(int)
    df_input['Year'] = df_input['Year'].astype(int)

    # = = = = = = = = = = = = = = =

    list_dict, grouped = [], df_input.groupby('Vehicle Url')
    for url, group in tqdm(grouped, desc='Progress', ncols=77):
        df_temp = group.reset_index(drop=True)

        dict_vehicle = {}
        for i in range(len(df_temp)):
            make, model, engine, year = df_temp.loc[i, 'Make'], df_temp.loc[i, 'Model'], df_temp.loc[i, 'Configuration'], df_temp.loc[i, 'Year']

            if make not in dict_vehicle:
                dict_vehicle[make] = {model: {engine: [year]}}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = {engine: [year]}
            elif engine not in dict_vehicle[make][model]:
                dict_vehicle[make][model][engine] = [year]
            elif year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)

        # = = = = = = = = = = = = = = =

        list_vehicle, list_vehicle_2 = [], []
        for make, dict_model in dict_vehicle.items():
            for model, dict_engine in dict_model.items():
                list_year = [year for list_year in dict_engine.values() for year in list_year]
                year_begin, year_end = min(list_year), max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

                for engine, list_year in dict_engine.items():
                    year_begin, year_end = min(list_year), max(list_year)
                    list_vehicle_2.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

        # = = = = = = = = = = = = = = =

        list_dict.append({'Vehicle Url': url,
                          'Vehicle W/o Engine': '\n'.join(sorted(list_vehicle)),
                          'Vehicle W/ Engine': '\n'.join(sorted(list_vehicle_2))})

    # = = = = = = = = = = = = = = =

    df_output = pd.DataFrame(list_dict).sort_values(by=['Vehicle Url'],
                                                    ascending=[True],
                                                    ignore_index=True)
    df_output.to_excel(f'{directory_out}/{file}', index=False)

print()
print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|█████████████████████████████| 92/92 [00:00<00:00, 599.83it/s]


Done ~
